In [2]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm as cm
import netCDF4
import numpy as np
import pandas as pd
import scipy as sci
import mpl_toolkits
from PIL import Image
from mpl_toolkits.basemap import Basemap


In [3]:
# 2009 - 2013 (monthly data)
ncs = [
    'GOSATTFTS2009060120091231_4AF04MV0103.nc',
    'GOSATTFTS2010010120101231_4AF04MV0103.nc',
    'GOSATTFTS2011010120111231_4AF04MV0103.nc',
    'GOSATTFTS2012010120121231_4AF04MV0103.nc',
    'GOSATTFTS2013010120130930_4AF04MV0103.nc'
]

filepath='./data/L4ACH4/{}'

nc_2009 = netCDF4.Dataset(filepath.format(ncs[0]))
nc_2010 = netCDF4.Dataset(filepath.format(ncs[1]))
nc_2011 = netCDF4.Dataset(filepath.format(ncs[2]))
nc_2012 = netCDF4.Dataset(filepath.format(ncs[3]))
nc_2013 = netCDF4.Dataset(filepath.format(ncs[4]))

In [6]:
nc_2013.data_production_date
print nc_2012.data_production_date
print nc_2011.data_production_date
print nc_2010.data_production_date
print nc_2009.data_production_date

2016-06-27
2016-01-08
2016-01-08
2016-01-08


In [7]:
def set_data(_data, _nc):
    """
    Return a dictionary with relevant informtion
        ant_flx_rgn_imp
        ant_flx_grd_imp
        lat [float]
        lon [float]
        time [int]
    """
    _data['data'] = {
        'flux_by_region': _nc.variables['ant_flx_rgn_imp'][:],
        'flux_by_grid':  _nc.variables['ant_flx_grd_imp'][:]
    }
    _data['lat'] = _nc.variables['lat'][:]
    _data['lon'] = _nc.variables['lon'][:]
    _data['time'] = _nc.variables['time'][:]
    return _data

## SET DATA

In [8]:
data_2013 = set_data({}, nc_2013) # most recent data
data_2012 = set_data({}, nc_2012)
data_2011 = set_data({}, nc_2011)
data_2010 = set_data({}, nc_2010)
data_2009 = set_data({}, nc_2009)

In [22]:
data_2013['data']['flux_by_grid'][0][34, 61]

cal_data = []


for month_data in data_2010['data']['flux_by_grid']:
    cal_data.append(month_data[34, 61])

for month_data in data_2011['data']['flux_by_grid']:
    cal_data.append(month_data[34, 61])

for month_data in data_2012['data']['flux_by_grid']:
    cal_data.append(month_data[34, 61])

for month_data in data_2013['data']['flux_by_grid']:
    cal_data.append(month_data[34, 61])

In [25]:
len(cal_data)

45

## Flatten DATA

In [9]:
# all data flattened per year
flat_data_2013 = data_2013['data']['flux_by_grid'].flatten()
flat_data_2012 = data_2012['data']['flux_by_grid'].flatten()
flat_data_2011 = data_2011['data']['flux_by_grid'].flatten()
flat_data_2010 = data_2010['data']['flux_by_grid'].flatten()
flat_data_2009 = data_2009['data']['flux_by_grid'].flatten()

#nov = get_monthly_region_data(data_2012, 11)

## Print NUM data points

In [ ]:
print "total number of data points for 12 months : {}".format(180*360*12)
print "total number of data points in 2012 grid data: {}".format(flat_data_2012.size)
print "total number of data points for 9 months: {}".format(180*360*9)
print "total number of data points in 2013 grid data: {}".format(flat_data_2013.size)

## Function to get data by month

In [ ]:
# monthly range [0, 11], exception of 2013 data which ranges from [0, 9]
get_monthly_grid_data = lambda d, m: d['data']['flux_by_grid'][m, :, :]

#Focuing on grid data
#get_monthly_region_data = lambda data, month: data['data']['flux_by_region'][month, :, :]

get_monthly_grid_data(data_2012, 10).flatten().sum()
#b=data_2009['data']['flux_by_grid']
#b.shape
#region_data =data_2012['data']['flux_by_region'][1,:,:]

In [ ]:
get_monthly_grid_data(data_2013, 0).shape

## Function to plot histogram

In [ ]:
def plot_hist(_plt, _data, year, x_range, y_max=50000, color_type='green'):
    """
    _plt [matplotlib.pyplot]
    _data [2D Numpy array] input data (already flattened)
    """

    min_val = x_range[0]
    max_val = x_range[1]
   
    plt.hist(_data, 100, [min_val, max_val], color=color_type, rwidth=0.9)

    # set x range
    plt.xlim([min_val, max_val])
    plt.ylim([0, y_max])
        
    # set labels and title
    plt.xlabel('Methane Concentration [{},{}]'.format(min_val, max_val))
    plt.ylabel('Count')
    
    plt.title('Methane Concentration Flux 1*1 Grid for {} [g m^-2 day^-1])'.format(year))
    
    # legend of zorro
    #plt.legend(('cdf','histogram ' + hist_type), loc = 'upper left')

def get_xrange(_data):
    return (np.amin(_data), np.amax(_data))

## Function to plot two histograms (normal range, >0.5)

In [ ]:
#def plot_hist(_plt, _data, hist_type, title='Histogram'):

def plot_hist2(_plt, _data, _year, color_type):

    # complete hist
    _plt.subplot(211)
    plot_hist(_plt, _data, year=_year, 
              x_range=get_xrange(flat_data_2013), 
              color_type=color_type)
    
    # ignoring < 0.5 (hist is skewed to left)
    _plt.subplot(212)
    plot_hist(_plt, _data, year=_year, 
              x_range=[0.5, 3.0], 
              y_max=20, 
              color_type=color_type)
    _plt.show()

## PLOT few sample histograms

In [ ]:
#plot_hist(plt, flat_data_2013, 'original', '2013 Methane Concentration')

plt.figure(figsize=(8, 4))
plot_hist2(plt, flat_data_2013, 2013, 'green')

plt.figure(figsize=(8, 4))
plot_hist2(plt, flat_data_2012, 2012, 'blue')

plt.figure(figsize=(8, 4))
plot_hist2(plt, flat_data_2009, 2009, 'purple')

## Print Annual Summed Concentration

In [ ]:
def print_annual_conc(year, _data):
    print "{} Annual value: {}".format(year, _data.sum())

In [ ]:
# Print all annual values in gm^-2 day^-1
print_annual_conc(2009, flat_data_2009)
print_annual_conc(2010, flat_data_2010)
print_annual_conc(2011, flat_data_2011)
print_annual_conc(2012, flat_data_2012)
print_annual_conc(2013, flat_data_2013)

In [ ]:
[get_monthly_grid_data(data_2009, m).sum() for m in xrange(7)]

In [ ]:
plt.contourf(data['lon'], data['lat'], get_monthly_grid_data(data_2013, 8), cmap=plt.cm.Purples)
plt.clim(0.5,2.0)
plt.ylim(-89.5, 89.5) # lat
plt.xlim(-179.5, 179.5) # lon

plt.colorbar()
plt.show()

In [ ]:
plt.contourf(data['lon'], data['lat'], nc.variables['ant_flx_grd_imp'][0], 
             cmap = plt.cm.PuOr_r,
             norm = matplotlib.colors.LogNorm()) # 0 for first month
plt.colorbar()
plt.show()

In [ ]:
nc.variables.keys()

In [ ]:
plt.contourf(data['lon'], data['lat'], nc.variables['ant_flx_grd_imp'][:].mean(axis=0)) # 0 for first month
plt.colorbar()
plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
# setup Lambert Conformal basemap.
# set resolution=None to skip processing of boundary datasets.
m = Basemap(width=12000000,height=9000000,projection='lcc',
            resolution=None,lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
m.etopo()
plt.show()

In [ ]:
m = Basemap(
    width=5000000, height=3400000, projection='aea', resolution='l',
    lat_1=29.5, lat_2=45.5, lat_0=38.5, lon_0=-96.)

In [ ]:
from mpl_toolkits.basemap import Basemap 
m = Basemap(projection='kav7',lon_0=0,resolution=None) 
map.drawcoastlines(linewidth=0.25) 
map.drawcountries(linewidth=0.25) 
map.fillcontinents(color='coral',lake_color='aqua')

## Trial A

resoultion is poor for California, find another map
Source?

In [ ]:
#-- create figure and axes instances
dpi = 100
fig = plt.figure(figsize=(1100/dpi, 1100/dpi), dpi=dpi)
ax  = fig.add_axes([0.1,0.1,0.8,0.9])

#-- create map
map = Basemap(projection='cyl', llcrnrlat= 30., urcrnrlat= 50.,\
              resolution='c',  llcrnrlon=-125.,urcrnrlon=-114.)

#-- draw coastlines, state and country boundaries, edge of map
map.drawcoastlines()
map.drawstates()
#map.drawcountries()

#-- create and draw meridians and parallels grid lines
#map.drawparallels(np.arange( -90., 90.,30.),labels=[1,0,0,0],fontsize=10)
#map.drawmeridians(np.arange(-180.,180.,30.),labels=[0,0,0,1],fontsize=10)

#-- convert latitude/longitude values to plot x/y values
x, y = map(*np.meshgrid(nc_2013['lon'],nc_2013['lat']))

#-- contour levels
clevs = np.arange(0.0,3.0,0.5)

# Jan Data
jan = get_monthly_grid_data(data_2013, 0)

#-- draw filled contours
cnplot = map.contourf(x,y, jan, clevs, cmap=plt.cm.Purples)

#-- add colorbar
cbar = map.colorbar(cnplot,location='bottom',pad="10%")      #-- pad: distance between map and colorbar
cbar.set_label('g m^-2 day^-1')                                      #-- add colorbar title string

#-- add plot title
plt.title('Methane')

#-- displa on screen
plt.show()

#-- maximize and save PNG file
#plt.savefig('plot_matplotlib_contour_filled_rect.png', bbox_inches='tight', dpi=dpi)


In [ ]:
print np.amin(jan)
print np.amax(jan)
jan.shape

In [ ]:
map = Basemap(projection='robin',
              resolution='c',
              lat_0=0,
              lon_0=0)

#datain = np.flipud( msk_array )
fig = plt.figure(figsize=(12,5))

nx = jan.shape[1]
ny = jan.shape[0]

xin = np.linspace(map.xmin, map.xmax, nx) # nx is the number of x points on the grid
yin = np.linspace(map.ymin, map.ymax, ny) # ny in the number of y points on the grid

lons = np.arange(-180,180,0.25) #from raster.GetGeoTransform()
lats  = np.arange(-90,90,0.25) 

lons, lats = np.meshgrid(lons,lats) 
xout, yout = map(lons, lats)
dataout = mpl_toolkits.basemap.interp(jan, xin, yin, xout, yout, order=1)

levels = np.arange(-2.0, 3.0, 0.5)
cntr = map.contourf(xout,yout,dataout, levels, cmap=cm.RdBu_r)
cbar = map.colorbar(cntr, location='bottom',pad='15%')

# Add some more info to the map
cstl = map.drawcoastlines(linewidth=.5)
meri = map.drawmeridians(np.arange(0,360,60), linewidth=.2, labels=[1,0,0,1], labelstyle='+/-', color='grey' ) 
para = map.drawparallels(np.arange(-90,90,30), linewidth=.2, labels=[1,0,0,1], labelstyle='+/-', color='grey')
boun = map.drawmapboundary(linewidth=0.5, color='grey')
plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap 
plt.
m = Basemap(projection='kav7',lon_0=0,resolution=None) 
map.drawcoastlines(linewidth=0.25) 
map.drawcountries(linewidth=0.25) 
map.fillcontinents(color='coral',lake_color='aqua')

In [ ]:
nc_2013['lat'][:]

In [ ]:
from mpl_toolkits.basemap import Basemap, cm
# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile
import numpy as np
import matplotlib.pyplot as plt

# plot rainfall from NWS using special precipitation
# colormap used by the NWS, and included in basemap.

nc = NetCDFFile('../../../examples/nws_precip_conus_20061222.nc')
# data from http://water.weather.gov/precip/

#prcpvar = nc.variables['amountofprecip']
#data = 0.01*prcpvar[:]
data = 0.01 * nc_2013['data']['flux_by_grid'][:]
latcorners = nc.variables['lat'][:]
loncorners = -nc.variables['lon'][:]
lon_0 = -nc.variables['true_lon'].getValue()
lat_0 = nc.variables['true_lat'].getValue()
# create figure and axes instances
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',
            lon_0=lon_0,
            lat_0=90.,
            lat_ts=lat_0,\
            llcrnrlat=latcorners[0],urcrnrlat=latcorners[2],\
            llcrnrlon=loncorners[0],urcrnrlon=loncorners[2],\
            rsphere=6371200.,resolution='l',area_thresh=10000)
# draw coastlines, state and country boundaries, edge of map.
m.drawcoastlines()
m.drawstates()
m.drawcountries()
# draw parallels.
parallels = np.arange(0.,90,10.)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,10.)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
ny = data.shape[0]; nx = data.shape[1]
lons, lats = m.makegrid(nx, ny) # get lat/lons of ny by nx evenly space grid.
x, y = m(lons, lats) # compute map proj coordinates.
# draw filled contours.
clevs = [0,1,2.5,5,7.5,10]
cs = m.contourf(x,y,data,clevs,cmap=cm.s3pcpn)
# add colorbar.
cbar = m.colorbar(cs,location='bottom',pad="5%")
cbar.set_label('mm')
# add title
plt.title('title')
plt.show()

In [ ]:
nc_2013.

In [ ]:
mp = Basemap(width=1284000,
             height=1164000,
             projection='lcc',
             lat_1=30.,
             lat_2=60,\
             lat_0=37,
             lon_0=-120.5,
             resolution='c', # lowest res
             rsphere=6370000.00)

# fill data: lats, lons, grid
cs = orb.pcolor(lons,lats,grid,cmap=cm.jet,norm=colors.LogNorm(),latlon=True)
mp.drawcounties()

In [ ]:
dd = get_monthly_grid_data(data_2013, 0) # January data

dd[:]

In [ ]:
(124.5 + 113.5)/2

In [ ]:
plt.close()
plt.figure(figsize=(14, 16))

def california_map(ax=None, lllat=31.5, urlat=42.5,
                   lllon=-124.5, urlon=-113.5):
    m = Basemap(ax=ax, projection='stere',
                lon_0=(urlon + lllon) / 2,
                lat_0=(urlat + lllat) / 2,
                llcrnrlat=lllat, urcrnrlat=urlat,
                llcrnrlon=lllon, urcrnrlon=urlon,
                resolution='i')
    m.drawstates()
    m.drawcountries()
    m.drawcoastlines(color='lightblue')
    m.drawlsmask(land_color='white', ocean_color='0.8', lakes=False)
    return m

# long and lat
lon = np.arange(-124.5, -113.5, 1.0)
lat = np.arange(31.5, 42.5, 1.0)

# Build the figure and ax objects
fig, ax = plt.subplots()
ax.set_title('Methane Concentration in California')
m= california_map()


#m.plot()
#predDf = predDf.sort('y_pred') # Useful for plotting
x, y = m(lon, lat)
dd = get_monthly_grid_data(data_2013, 0) # January data
#serieA = (np.array(predDf['y_pred']) - predDf['y_pred'].min())/(predDf['y_pred'].max()-predDf['y_pred'].min())
z = plt.cm.jet(dd)
z = np.array(dd)

#im = m.pcolormesh(lon, lat, dd, cmap=plt.cm.jet, latlon=True)

#m.scatter(x, y, c=z, s=60, alpha=0.5, edgecolors='none')
#c = m.colorbar(location='right')
#c.set_label("House Value (Thousands of $)")
#m.plot()
plt.show()

In [ ]:
# Just the contingous United States
m = Basemap(
    width=50000000, 
    height=34000000, 
    projection='aea', 
    resolution='c',
    lat_0=35.5, 
    llcrnrlat=32.0, urcrnrlat=42.5,
    llcrnrlon=-124.5, urcrnrlon=-113.5,
    lon_0=-119.0)

In [ ]:
# Build the figure and ax objects
fig, ax = plt.subplots(figsize=(18, 8))
ax.set_title('PRISM 2011 Tmax')
 
# Built-in map boundaries
m.drawcoastlines(color='dodgerblue')
m.drawlsmask(land_color='white', ocean_color='0.8', lakes=False)
m.drawcountries()
m.drawstates()
plt.show()

In [ ]:
# Build arrays of the lat/lon points for each cell
lon = np.arange(-124.5, -113.5, 1.0)
lat = np.arange(31.5, 42.5, 1.0)
lons, lats = np.meshgrid(lons, lats)